# Web Scraping

 ### Inhalte
 
 - Was ist Web Scraping? 
 - Die Grundlagen von Webseiten verstehen
 - Einen Scraper in Python bauen


# Was ist Web Scraping?

**to scrape:** etw. abkratzen, schaben, schürfen

Unter Web Scraping versteht man das Extrahieren (oder Abkratzen) von Inhalten und Daten von Webseiten. 


Ein Scraper ist ein Programm (Python-Skript) mit zwei Kernaufgaben:

- dem Aufrufen der gewünschten Webseiten
- die Extraktion und Ablage der relevanten Informationen


![API](../../imgs/scraping.png)
Quelle: https://kinsta.com/de/wissensdatenbank/was-ist-web-scraping/




## Web Crawling vs. Web Scraping

![Crawler vs Scraper](../../imgs/crawl_vs_scrape.png)
![Crawler vs Scraper](../../imgs/crawler.png)

Quellen:

https://limeproxies.netlify.app/blog/crawling-vs-scraping

https://dev.to/soax/web-crawling-vs-web-scraping-what-is-the-difference-4chl


## Wann benutzen wir Scraper?

- Wenn die gewünschten Daten nicht verfügbar sind:
    - keine Datenfiles (csv, json, excel, etc.)
    - keine API Schnittstelle
- Wenn die Informationen auf einer (oder mehreren) Webseiten
    - strukturiert abrufbar sind
    - nicht hinter Paywalls, Passwörtern oder Captchas versteckt sind


## Ist Scrapen legal?

Die kurze Antwort: **Meistens ja, aber es kommt drauf an.**

    
Öffentlich zugängliche Daten, also Inhalte ohne "Schutzvorrichtungen", zu sammeln ist legal unter Berücksichtigung des  Urheberrechts und bei personenbezogenen Daten auch der DSGVO.


Quelle: https://www.computerwoche.de/a/was-ist-scraping,3551081




## Robots.txt

Viele Webseiten veröffentlichen eine *robots.txt* Datei. Diese zeigt an ob und wie Bots auf der Seite erlaubt sind.

Beispielsweise:

- https://www.uni-leipzig.de/robots.txt
- https://www.airbnb.com/robots.txt


# Die Grundlagen von Webseiten verstehen

Die Grundlage einer Webseite besteht aus einem **Hypertext Markup Language (HTML)** File.
Dieses teilt die Seite in Abschnitte ein und definiert welche Textabschnitte Überschriften oder Paragraphen sind.


Mit **Cascading Style Sheets (CSS)** wird das Design festgelegt, beispielsweise die Schriftart und Schriftfarbe oder die Abstände zwischen Textelementen.

Zusätzlich wird **JavaScript** verwendet um PopUps oder interaktive Elemente auf eine Webseite einzufügen.


Die Entwicklertools im Browser helfen eine Seite zu verstehen.


# Einen Scraper bauen

In [ ]:
!pip install requests
import requests

In [ ]:
url = 'https://www.mdr.de/nachrichten/sachsen/index.html'
response = requests.get(url)

In [ ]:
response

In [ ]:
dir(response)

In [ ]:
response.status_code

In [ ]:
response.json()

In [ ]:
response.text

In [ ]:
type(response.text)

In [ ]:
'Wetter' in response.text

In [ ]:
response.text.find("title")

In [ ]:
response.text[170:250]

# BeautifulSoup

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

Dokumentation: https://beautiful-soup-4.readthedocs.io/en/latest/

In [ ]:
!pip3 install beautifulsoup4

In [ ]:
import requests 
from bs4 import BeautifulSoup

results = BeautifulSoup(response.text, 'html.parser')

In [ ]:
results

In [ ]:
type(results)

In [ ]:
dir(results)

### Nachrichtenblöcke finden

In [ ]:
teaser = results.find_all('div', class_='teaser')

In [ ]:
teaser

In [ ]:
teaser = results.find_all('a', class_='linkAll')

In [ ]:
teaser

In [ ]:
type(teaser)

In [ ]:
len(teaser)

In [ ]:
teaser[0]

In [ ]:
teaser[1]

## Aufgabe

Finde alle Überschriften (Headlines) auf der Seite

In [ ]:
headlines

In [ ]:
headlines[0]

In [ ]:
type(headlines[0])

## Beautiful Soup Tag Element

https://beautiful-soup-4.readthedocs.io/en/latest/#kinds-of-objects

In [ ]:
dir(headlines[0])

In [ ]:
headlines[0].text

## Aufgabe

Iteriere durch alle Headlines und gibt den Text aus

## Duplikate entfernen

In [ ]:
# Duplikate entfernen
titel = []
for headline in headlines:
    clean_headline = headline.text.strip()
    if clean_headline not in titel:
        titel.append(clean_headline)

# Titel ausgeben        
for elem in titel:
    print(elem)

## Auf Links zugreifen

In [ ]:
headlines

In [ ]:
headlines[0].get('title')

In [ ]:
headlines[0].get('href')

## Titel und Link in Dictionary speichern

In [ ]:
data = []
titel = []
for headline in headlines:
    clean_headline = headline.text.strip()
    if clean_headline not in titel:
        current_data = {'titel': clean_headline,
                        'link': headline.get('href')}
        data.append(current_data)
        titel.append(clean_headline)

In [ ]:
# Einzelnen Elemente ausgeben
for elem in data:
    print(elem)

In [ ]:
data[0]

## Daten in Dataframe umwandeln und speichern

In [ ]:
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.to_csv('nachrichten_sachsen.csv')

## Informationen auf den einzelnen Seiten holen

In [ ]:
data[0]['link']

In [ ]:
base_url = "https://www.mdr.de/"
url = base_url + data[0]['link']

In [ ]:
print(url)

In [ ]:
response = requests.get(url)
result = BeautifulSoup(response.text, 'html.parser')

In [ ]:
result

### Den Autor extrahieren

In [ ]:
author = result.find('p', class_='author').text

In [ ]:
author

### Das Veröffentlichungsdatum extrahieren

In [ ]:
webtime = result.find('p', class_='webtime')

In [ ]:
dateinfo = webtime.select('span')

In [ ]:
dateinfo

In [ ]:
dateinfo[1].text

In [ ]:
full_date = dateinfo[1].text + dateinfo[2].text

## Abstecher: Timestamps in Python

String Datum in Datetime umwandeln.

Module: Datetime
 - https://docs.python.org/3/library/datetime.html
 
Funktion: Strptime
 https://www.geeksforgeeks.org/python-datetime-strptime-function/

In [ ]:
from datetime import datetime
import locale

locale.setlocale(locale.LC_ALL, 'de_DE.utf8')

datetime.strptime(dateinfo[1].text, '%d. %B %Y, ')

In [ ]:
full_date

In [ ]:
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.utf8')

def create_timestamp(elements):
    date = elements[1].text
    time = elements[2].text
    return datetime.strptime(date + time, '%d. %B %Y, %H:%M Uhr')

In [ ]:
create_timestamp(dateinfo)

## Aufgabe:

Extrahiere von allen gespeicherten Links die Autoren und das zugehörige Veröffentlichungsdatum